Run the notebook at TODAY after 15:00:00 (so the market is closed and data is available).

Input your current position.

The orders are for tomorrow.

In [1]:
import pandas as pd
cal = pd.read_csv('/data/linq/.qlib/qlib_data/cn_data/calendars/day.txt')
TODAY = str(cal.iloc[-1,0])
YESTERDAY = str(cal.iloc[-2,0])
# TODAY = str(cal.iloc[-2,0])
# YESTERDAY = str(cal.iloc[-3,0])

BT_START = '2024-12-01'

print(BT_START)
print(YESTERDAY)
print(TODAY)

2024-12-01
2024-12-09
2024-12-10


In [2]:
# BT_START = '2024-12-01'
# TODAY = '2024-12-09'

test_split = (TODAY, TODAY)
bt_split = (BT_START, YESTERDAY)
pos_txt = f'''\
date {TODAY}
cash 336.66
SH600570 800
SH601698 900
SH600104 1100
SH600588 1500
SH600233 1000
'''



REC_LIST = [
    ('GBDT0', 'EXP_BENCH', 'c745fed5a5ec439f9abeafac76e750a5'), # GBDT BENCH_NOW r0 all
    ('GBDT1', 'EXP_BENCH', 'ffe9470bec09417fb4db91128edc0a87'), # GBDT BENCH_NOW r1 all
    ('GBDT2', 'EXP_BENCH', 'e420ff30870d4e55a1c5e0fb207fa8a0'), # GBDT BENCH_A r0 all
    # ('GBDT3', 'EXP_BENCH', '1dcc7b1ec14e46379690c4f2d6e5242d'), # GBDT BENCH_A r1 all
    ('GBDT4', 'EXP_BENCH', '8d7e72af7f4b475f9f3646021a29bb1e'), # GBDT BENCH_B r0 all
    # ('GBDT5', 'EXP_BENCH', 'd308acba0eaf453b9f5acfc309b05c46'), # GBDT BENCH_B r1 all
    ('GBDT6', 'EXP_BENCH', 'ec4284016f6d44d897969410a5ef3ec0'), # GBDT BENCH_NOW r0 csi300
    # ('GBDT7', 'EXP_BENCH', 'c9dc8e870efc459396ea77850a7bc099'), # GBDT BENCH_NOW r1 csi300
    ('GBDT8', 'EXP_BENCH', '1f1711ba5caa4fd7a298196fd6a57558'), # GBDT BENCH_A r0 csi300
    # ('GBDT9', 'EXP_BENCH', 'a79446d3724942c58ed14c59622fbe2c'), # GBDT BENCH_A r1 csi300
    ('GBDT10', 'EXP_BENCH', '74d88afd5cb3448d831402b4e7571dc6'), # GBDT BENCH_B r0 csi300
    # ('GBDT5', 'EXP_BENCH', '58317fb6231c4ba0b4d8cecc07db822d'), # GBDT BENCH_B r1 csi300
]

# EXP_NAME, rid = REC_LIST[0]
TOPK = 5
NDROP = 1
HT = 0

info = {
    'ALGO': ['GBDT'],
    'market': ['csi300_ext'], 
    # 'market': ['all'], 
    'benchmark': ["SH000300"], 
    'feat': ["Alpha158"], 
    # 'feat': ["Alpha360"], 
    'label': ['r0'],
    'params': [f'topk {TOPK} ndrop {NDROP} HT {HT}']
}

In [3]:
import qlib
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2815487:MainThread](2024-12-11 17:57:44,885) INFO - qlib.Initialization - [config.py:416] - default_conf: client.


[2815487:MainThread](2024-12-11 17:57:44,889) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.


[2815487:MainThread](2024-12-11 17:57:44,889) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


Record position history

In [4]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
hold_lst = pos.get_stock_list()
print(dt)
print(account)
print(norm_pos)
print(hold_lst)

2024-12-10 09:00:00
{'SH600570': {'amount': 800}, 'SH601698': {'amount': 900}, 'SH600104': {'amount': 1100}, 'SH600588': {'amount': 1500}, 'SH600233': {'amount': 1000}, 'cash': 336.66}
{'_settle_type': 'None', 'position': {'SH600588': {'amount': 7188, 'price': 2.6728668212890625}, 'SH601698': {'amount': 3497, 'price': 5.913832664489746}, 'SH600570': {'amount': 3727, 'price': 6.836996555328369}, 'SH600233': {'amount': 7140, 'price': 2.0420126914978027}, 'SH600104': {'amount': 12461, 'price': 1.586225152015686}, 'cash': 336.66, 'now_account_value': 100057.30793741703}, 'init_cash': 336.66}
['SH600588', 'SH601698', 'SH600570', 'SH600233', 'SH600104']


Prepare dataset

In [5]:
nameDFilter = NameDFilter(name_rule_re='(SH60[0-9]{4})|(SZ00[0-9]{4})')
filter_pipe=[nameDFilter]
benchmark = BENCH_Step(test_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

bt_benchmark = BENCH_Step(bt_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

In [6]:
dataset = init_instance_by_config(benchmark.dataset_config)
bt_dataset = init_instance_by_config(bt_benchmark.dataset_config)
# dataset.prepare('test')

[2815487:MainThread](2024-12-11 17:58:31,388) INFO - qlib.timer - [log.py:127] - Time cost: 46.006s | Loading data Done


[2815487:MainThread](2024-12-11 17:58:31,392) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | DropnaLabel Done


[2815487:MainThread](2024-12-11 17:58:31,397) INFO - qlib.timer - [log.py:127] - Time cost: 0.004s | CSZScoreNorm Done


[2815487:MainThread](2024-12-11 17:58:31,398) INFO - qlib.timer - [log.py:127] - Time cost: 0.009s | fit & process data Done


[2815487:MainThread](2024-12-11 17:58:31,399) INFO - qlib.timer - [log.py:127] - Time cost: 46.017s | Init data Done


[2815487:MainThread](2024-12-11 17:59:16,941) INFO - qlib.timer - [log.py:127] - Time cost: 45.540s | Loading data Done


[2815487:MainThread](2024-12-11 17:59:16,947) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | DropnaLabel Done


[2815487:MainThread](2024-12-11 17:59:16,959) INFO - qlib.timer - [log.py:127] - Time cost: 0.011s | CSZScoreNorm Done


[2815487:MainThread](2024-12-11 17:59:16,960) INFO - qlib.timer - [log.py:127] - Time cost: 0.017s | fit & process data Done


[2815487:MainThread](2024-12-11 17:59:16,961) INFO - qlib.timer - [log.py:127] - Time cost: 45.561s | Init data Done


In [7]:
label_df = bt_dataset.prepare("test")
label_df = pd.DataFrame(label_df.loc[:, 'LABEL0'])
label_df.columns = ['label']
# label_df.index = label_df.index.droplevel(0)
label_df

label
datetime   instrument          
2024-12-02 SH600000    0.016860
           SH600009   -0.000560
           SH600010   -0.005051
           SH600011    0.005666
           SH600015    0.030956
...                         ...
2024-12-09 SZ002841   -0.002499
           SZ002916   -0.001414
           SZ002920    0.018372
           SZ002938    0.031052
           SZ003816   -0.004902

[1542 rows x 1 columns]

Load and run model

In [8]:
def simp_eval(res_df, k, t=None, day=None):
    if day:
        date = str(cal.iloc[-day,0])
        res_df = res_df[res_df.index.get_level_values('datetime') >= date]
    # Step 1: Group by multiple indices
    grouped = res_df.groupby(['datetime'])

    # Step 2: Sort by 'value' within each group in descending order
    sorted_df = grouped.apply(lambda x: x.sort_values(by='score', ascending=False))
    sorted_df['win'] = sorted_df['label'] * sorted_df['score'] > 0
    # Step 3: Select the top 5 rows for each group
    top_k_per_group = sorted_df.groupby(['datetime']).head(k)
    if t:
        top_k_per_group = top_k_per_group.tail(t)
    # print(top_k_per_group)
    top_k_ic = top_k_per_group['label'].corr(top_k_per_group['score'])
    top_k_win = 1.0 * top_k_per_group['win'].sum() / len(top_k_per_group)
    return top_k_ic, top_k_win

def run_model(model):
    BT_EXP_NAME = 'backtest'

    # backtest and analysis
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, bt_dataset, recorder)
        sr.generate()
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        bt_pred_df = recorder.load_object("pred.pkl")
    res_df = pd.concat([label_df, bt_pred_df], axis=1, sort=True).reindex( label_df.index)

    def eval_day(day=None):
        top5ic, top5win = simp_eval(res_df, 5, day=day)
        top10ic, top10win = simp_eval(res_df, 10, 5, day=day)
        top15ic, top15win = simp_eval(res_df, 15, 5, day=day)
        top20ic, top20win = simp_eval(res_df, 20, 5, day=day)
        topic, topwin = simp_eval(res_df, 20, day=day)
        return [topic, topwin, top5ic, top5win, top10ic, top10win, top15ic, top15win, top20ic, top20win]
    day5metrics = eval_day(5)
    day10metrics = eval_day(10)
    day20metrics = eval_day(20)
    EXP_NAME = 'realworld_position_maker'

    # backtest and analysis
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, dataset, recorder)
        sr.generate()
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        pred_df = recorder.load_object("pred.pkl")

    return day5metrics, day10metrics, day20metrics, pred_df

# Run

In [9]:
info_dict5 = {}
info_dict10 = {}
info_dict20 = {}
pred_lst = []

for NAME, EXP_NAME, rid in REC_LIST:
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=rid)
        model = recorder.load_object("trained_model")
    day5metrics, day10metrics, day20metrics, pred_df = run_model(model)
    info_dict5[NAME] = day5metrics
    info_dict10[NAME] = day10metrics
    info_dict20[NAME] = day20metrics
    pred_lst.append(pred_df)

[2815487:MainThread](2024-12-11 17:59:17,017) INFO - qlib.workflow - [exp.py:258] - Experiment 2 starts running ...


[2815487:MainThread](2024-12-11 17:59:17,105) INFO - qlib.workflow - [recorder.py:341] - Recorder 5d1ece9de1e7484ebbc1c8459b2c3137 starts running under Experiment 2 ...


[2815487:MainThread](2024-12-11 17:59:19,170) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


[2815487:MainThread](2024-12-11 17:59:19,177) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...


[2815487:MainThread](2024-12-11 17:59:19,188) INFO - qlib.workflow - [recorder.py:341] - Recorder 506f3ba71f4a49cd8f7f18e1143e2ca8 starts running under Experiment 4 ...


...

[2815487:MainThread](2024-12-11 17:59:29,939) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


[2815487:MainThread](2024-12-11 17:59:29,945) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...


[2815487:MainThread](2024-12-11 17:59:29,959) INFO - qlib.workflow - [recorder.py:341] - Recorder 53c84deb23b54c349c3802fe9d17a952 starts running under Experiment 3 ...


'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-12-10 SH600000    0.030046
           SH600009    0.097639
           SH600010   -0.066421
           SH600011    0.001421
           SH600015   -0.307438


[2815487:MainThread](2024-12-11 17:59:30,232) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


In [10]:
info_df5 = pd.DataFrame(info_dict5, index=['TOP20IC', 'TOP20WIN', '0-5IC', '0-5WIN', '5-10IC', '5-10WIN', '10-15IC', '10-15WIN', '15-20C', '15-20WIN'])
report_info5 = info_df5

info_df10 = pd.DataFrame(info_dict10, index=['TOP20IC', 'TOP20WIN', '0-5IC', '0-5WIN', '5-10IC', '5-10WIN', '10-15IC', '10-15WIN', '15-20C', '15-20WIN'])
report_info10 = info_df10

info_df20 = pd.DataFrame(info_dict20, index=['TOP20IC', 'TOP20WIN', '0-5IC', '0-5WIN', '5-10IC', '5-10WIN', '10-15IC', '10-15WIN', '15-20C', '15-20WIN'])
report_info20 = info_df20

In [11]:
pred_df = pd.concat(pred_lst, axis=1)
pred_df.columns = name_lst = [r[0] for r in REC_LIST]
_pred_df = pred_df

pred_df['mean'] = _pred_df.mean(axis=1)
pred_df['std'] = _pred_df.std(axis=1)
pred_df['score'] = pd.concat([pred_df[n]*info_dict20[n][0] for n in name_lst], axis=1).sum(axis=1)
# pred_df

In [12]:
rank_df = pred_df.droplevel(0).sort_values(by='score', ascending=False)
fields = ['$close/$factor']
_df = D.features(rank_df.index.values, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['close']
_df = _df.droplevel(1)
rank_df = pd.concat([rank_df,_df],axis=1,join='inner')
report_top = rank_df.head(20)

In [13]:
hold_lst

['SH600588', 'SH601698', 'SH600570', 'SH600233', 'SH600104']

In [14]:
pos_rank = rank_df.sort_values(by='score', ascending=True)
pos_rank = pos_rank[pos_rank.index.isin(hold_lst)]
report_pos = pos_rank

In [15]:
sty_report_info5 = report_info5.style.background_gradient(axis=0)
sty_report_info5

,GBDT0,GBDT1,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10
TOP20IC,-0.181862,-0.006492,-0.177584,-0.125325,-0.166531,-0.027349,-0.175386
TOP20WIN,0.500000,0.612500,0.487500,0.525000,0.600000,0.575000,0.562500
0-5IC,-0.146310,0.190481,-0.293071,-0.123586,-0.285351,-0.100758,-0.053436
0-5WIN,0.350000,0.400000,0.450000,0.450000,0.450000,0.450000,0.500000
5-10IC,0.108726,-0.097320,0.067952,-0.774278,-0.618396,0.532151,-0.810227
5-10WIN,0.600000,0.600000,0.800000,0.200000,0.400000,0.400000,0.600000
10-15IC,0.163235,0.485844,0.168397,0.169377,0.014806,0.469000,-0.499942
10-15WIN,0.400000,0.800000,0.400000,0.800000,0.800000,1.000000,0.800000
15-20C,0.884893,0.531707,0.795316,0.080353,0.893108,0.396331,-0.434557
15-20WIN,0.600000,1.000000,0.600000,0.800000,0.800000,0.600000,0.600000


In [16]:
sty_report_info10 = report_info10.style.background_gradient(axis=0)
sty_report_info10

,GBDT0,GBDT1,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10
TOP20IC,0.300980,0.177116,0.328541,0.337494,0.238341,0.229248,0.164809
TOP20WIN,0.500000,0.533333,0.491667,0.508333,0.541667,0.583333,0.500000
0-5IC,0.580581,0.537463,0.531537,0.490091,0.506677,0.501410,0.354426
0-5WIN,0.366667,0.366667,0.500000,0.466667,0.366667,0.433333,0.433333
5-10IC,0.108726,-0.097320,0.067952,-0.774278,-0.618396,0.532151,-0.810227
5-10WIN,0.600000,0.600000,0.800000,0.200000,0.400000,0.400000,0.600000
10-15IC,0.163235,0.485844,0.168397,0.169377,0.014806,0.469000,-0.499942
10-15WIN,0.400000,0.800000,0.400000,0.800000,0.800000,1.000000,0.800000
15-20C,0.884893,0.531707,0.795316,0.080353,0.893108,0.396331,-0.434557
15-20WIN,0.600000,1.000000,0.600000,0.800000,0.800000,0.600000,0.600000


In [17]:
sty_report_info20 = report_info20.style.background_gradient(axis=0)
sty_report_info20

,GBDT0,GBDT1,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10
TOP20IC,0.300980,0.177116,0.328541,0.337494,0.238341,0.229248,0.164809
TOP20WIN,0.500000,0.533333,0.491667,0.508333,0.541667,0.583333,0.500000
0-5IC,0.580581,0.537463,0.531537,0.490091,0.506677,0.501410,0.354426
0-5WIN,0.366667,0.366667,0.500000,0.466667,0.366667,0.433333,0.433333
5-10IC,0.108726,-0.097320,0.067952,-0.774278,-0.618396,0.532151,-0.810227
5-10WIN,0.600000,0.600000,0.800000,0.200000,0.400000,0.400000,0.600000
10-15IC,0.163235,0.485844,0.168397,0.169377,0.014806,0.469000,-0.499942
10-15WIN,0.400000,0.800000,0.400000,0.800000,0.800000,1.000000,0.800000
15-20C,0.884893,0.531707,0.795316,0.080353,0.893108,0.396331,-0.434557
15-20WIN,0.600000,1.000000,0.600000,0.800000,0.800000,0.600000,0.600000


In [18]:
color_lst = [n for n in report_pos.columns if n not in ['close']]
# color_lst

In [19]:
sty_report_pos = report_pos.style.background_gradient(cmap='Blues_r', subset=color_lst)
sty_report_pos

,GBDT0,GBDT1,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,,
SH600104,-0.057261,0.049873,-0.188971,-0.141267,-0.053811,-0.118645,-0.236531,-0.106659,0.088414,-0.197169,17.969999
SH601698,-0.068132,-0.026254,-0.129749,-0.108736,-0.027927,-0.099029,-0.257208,-0.102434,0.072981,-0.176231,22.980001
SH600570,-0.053414,0.019185,-0.113095,-0.082817,-0.019608,-0.097395,-0.150231,-0.071053,0.053412,-0.129546,31.860001
SH600233,0.010960,0.022529,-0.041270,-0.000023,0.022817,-0.005509,-0.062517,-0.007573,0.030211,-0.012405,14.579999
SH600588,0.023668,0.025258,-0.030108,0.039939,0.002404,-0.004383,0.022337,0.011302,0.021809,0.018434,12.809999


In [20]:
sty_report_top0 = report_top.head(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top1 = report_top.tail(10).style.background_gradient(axis=0, subset=color_lst)

In [21]:
sty_report_top0

,GBDT0,GBDT1,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,,
SH600029,0.224671,0.034080,0.291393,0.292746,0.324759,0.487245,0.693309,0.335458,0.192012,0.571559,7.240000
SH601111,0.192957,0.023636,0.240976,0.237303,0.201021,0.313656,0.419305,0.232693,0.111967,0.410443,8.889999
SH603259,0.153475,0.024361,0.258174,0.248628,0.187975,0.308314,0.389704,0.224376,0.108541,0.398949,57.970001
SH600732,0.154157,0.081897,0.225593,0.349532,0.126354,0.190885,0.351731,0.211450,0.097597,0.384829,13.400000
SH601336,0.108419,0.005988,0.218931,0.234564,0.233191,0.319403,0.400072,0.217224,0.120055,0.379521,53.220001
SH600115,0.133521,0.028577,0.232096,0.139743,0.230796,0.272944,0.543047,0.225818,0.150119,0.375743,4.330000
SH600989,0.071717,0.017128,0.174244,0.077632,0.166450,0.186888,0.213909,0.129710,0.068078,0.225836,16.499998
SH600438,0.089579,0.072412,0.122652,0.135199,0.137930,0.136440,0.186658,0.125839,0.034277,0.220628,25.260000
SZ002841,0.079839,0.020259,0.129672,0.090617,0.114294,0.171469,0.282539,0.126956,0.076793,0.213918,39.919998


In [22]:
sty_report_top1

,GBDT0,GBDT1,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,,
SH600886,0.101738,0.033103,0.166612,0.055300,0.094989,0.136766,0.129742,0.102607,0.043305,0.185263,15.810000
SZ002311,0.049666,0.001540,0.086157,0.112179,0.089682,0.131945,0.287996,0.108452,0.083270,0.180474,47.529999
SZ002821,0.055752,0.005660,0.086899,0.098144,0.107033,0.112434,0.200960,0.095269,0.055127,0.163862,85.300003
SH600027,0.045796,0.019513,0.127813,0.056101,0.100520,0.092540,0.221793,0.094868,0.061901,0.159891,5.620000
SZ002236,0.042879,0.000033,0.099081,0.039391,0.149698,0.166911,0.139968,0.091137,0.059672,0.155769,17.029999
SH601865,0.037308,0.058864,0.033373,0.085236,0.098282,0.136272,0.129675,0.082716,0.038522,0.137422,24.070000
SH600426,0.027160,0.014406,0.074199,0.031392,0.098164,0.132000,0.150657,0.075426,0.049815,0.124185,22.879999
SH603195,0.031503,0.016041,0.075884,0.043954,0.067309,0.102344,0.185858,0.074699,0.052683,0.122224,73.859993
SH600660,0.051599,0.018737,0.059242,0.093937,0.047954,0.073007,0.141462,0.069420,0.036361,0.121496,56.599998


In [23]:
import dataframe_image as dfi

infodf = pd.DataFrame({'label': ['Model update date', 'Prediction generation date'],
                       'date': ['2024-12-11', TODAY]})

dfi.export(infodf, './tmp/ensemble_info.png',table_conversion='matplotlib')

dfi.export(sty_report_info5, './tmp/ensemble_performance5.png',table_conversion='matplotlib')
dfi.export(sty_report_info10, './tmp/ensemble_performance10.png',table_conversion='matplotlib')
dfi.export(sty_report_info20, './tmp/ensemble_performance20.png',table_conversion='matplotlib')
dfi.export(sty_report_top0, './tmp/ensemble_top0.png',table_conversion='matplotlib')
dfi.export(sty_report_top1, './tmp/ensemble_top1.png',table_conversion='matplotlib')
dfi.export(sty_report_pos, './tmp/ensemble_watch.png',table_conversion='matplotlib')